# NEON pre-processor notebook

##### Author : Negin Sobhani negins@ucar.edu [@negin513](https://github.com/negin513)

##### Last revised: 2022-10-11
_______

This is a notebook for reading neon simulation and evaluation files and save them as csv files. This would be next used for creating statistics and spatial plots. 

In [1]:
import os
import time
import datetime

import numpy as np
import pandas as pd
import xarray as xr

from glob import glob
from os.path import join

import tqdm
import cftime

In [2]:
print('xarray '+xr.__version__) ##-- was working with 0.20.0

xarray 0.20.2


## Dask Cluster
The following will spin up a dask cluster,  only works on NCAR machines. 
16 workers worked best for NEON.

In [3]:
from distributed import Client
from ncar_jobqueue import NCARCluster

cluster = NCARCluster()
cluster.scale(16)
client = Client(cluster)
client

/glade/u/apps/opt/conda/envs/npl/lib/python3.7/site-packages/dask_jobqueue/core.py:20: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile


Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/wwieder/new_server/proxy/8787/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/wwieder/new_server/proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.12.206.48:41164,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/wwieder/new_server/proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


---------------------------

In [4]:
# -- read list of sites:
#neon_sites_pft = pd.read_csv('/glade/scratch/negins/preprocessed_neon_csv_data/neon_sites_dompft.csv')
neon_sites_pft = pd.read_csv('/glade/scratch/wwieder/run_NEON_v2/tools/site_and_regional/neon_sites_dompft.csv')
neon_sites = neon_sites_pft['Site'].to_list()

failed_sites = [] # -- list for saving failed sites

### Download evaluation data
This is slow and only has to be done once!

In [5]:
# -- read only these variables from the whole netcdf files

def preprocess (ds):
    '''
    reading selected variables from ctsm outputs
    '''
    variables = ['FCEV', 'FCTR', 'FGEV','FSH','GPP','FSA','FIRA','AR','HR','ELAI']

    ds_new= ds[variables]
    return ds_new

In [6]:
# -- fix timestamp on CTSM files so they can be matched with eval files

def fix_time (ds):
    '''
    fix time formatting with reading multiple cesm files.
    '''
    nsteps = len(ds.time)
    yr0 = ds['time.year'][0].values
    month0 = ds['time.month'][0].values
    day0 = ds['time.day'][0].values

    date = cftime.datetime(yr0,month0,day0).isoformat() 
    ds['time'] = xr.cftime_range(date, periods=nsteps, freq='30min')
    ds['time']= ds['time'].dt.strftime("%Y-%m-%d %H:%M:%S")
    return ds

In [7]:
#-- path to save the csv files:
csv_dir = "/glade/scratch/wwieder/preprocessed_neon_v2/"

#create the directory if it does not exist:
if not os.path.isdir(csv_dir):
    os.mkdir(csv_dir)

In [9]:
years = ["2018","2019","2020","2021"]

for neon_site in neon_sites:
    try: 

        start_site = time.time()

        print ('---------------------------')
        print ("Extracting data for "+neon_site)
        sim_files =[]
        for year in years:
            sim_path = "/glade/scratch/wwieder/NEON_testV2/archive/"+neon_site+".transient/lnd/hist/"
            #sim_path = "/glade/scratch/wwieder/run_NEON_v2/tools/site_and_regional/archive/"+neon_site+".transient/lnd/hist/"
            #sim_path = "/glade/scratch/negins/neon_v2/tools/site_and_regional/archive/"+neon_site+".transient/lnd/hist/"
            sim_files.extend(sorted(glob(join(sim_path,neon_site+".transient.clm2.h1."+year+"*.nc"))))

        print("All simulation files for all years: [", len(sim_files), "files]")

        start = time.time()

        ds_ctsm = xr.open_mfdataset(sim_files, decode_times=True, combine='by_coords',parallel=True,preprocess=preprocess)
        ds_ctsm = fix_time (ds_ctsm)

        end = time.time()
        print("Reading all simulation files took:", end-start, "s.")

        eval_files = []
        for year in years:
            #eval_dir = "/glade/work/negins/neon_scripts/notebooks/evaluation_files_2/"+neon_site
            eval_dir = "/glade/work/wwieder/neon/eval_files/"+neon_site
            eval_files.extend(sorted(glob(join(eval_dir,neon_site+"_eval_"+year+"*.nc"))))


        print ("All evaluation files for all years: [", len(eval_files), "files]")

        start = time.time()

        ds_eval = xr.open_mfdataset(eval_files, decode_times=True, combine='by_coords')

        end = time.time()
        print("Reading all observation files took:", end-start, "s.")
        
        #Convert CTSM data to a Pandas Dataframe for easier handling:
        ctsm_vars = ['FCEV', 'FCTR', 'FGEV','FSH','GPP','FSA','FIRA','AR','HR','ELAI']

        df_ctsm = pd.DataFrame({'time':ds_ctsm.time})
        df_ctsm['time'] = pd.to_datetime(df_ctsm['time'],format= '%Y-%m-%d %H:%M:%S' )

        for var in tqdm.tqdm(ctsm_vars):
            sim_var_name = "sim_"+var
            field = np.ravel ( ds_ctsm[var])     
            df_ctsm[sim_var_name]=field
            #-- shift simulation data by one
            df_ctsm[sim_var_name]=df_ctsm[sim_var_name].shift(-1).values


        #Convert NEON data to a Pandas Dataframe for easier handling:
        eval_vars = ['NEE','FSH','EFLX_LH_TOT','GPP','Rnet']

        df_all = pd.DataFrame({'time':ds_eval.time})

        for var in eval_vars:
            field = np.ravel (ds_eval[var])
            df_all[var]=field
        
        # -- merge two pandas dataframe on time
        df_all=df_all.merge(df_ctsm.set_index('time'), on='time', how='left')

        clm_var = 'sim_EFLX_LH_TOT'
        # -- Latent Heat Flux:
        # -- EFLX_LH_TOT = FCEV + FCTR +FGEV
        df_all [clm_var] = df_all['sim_FCEV']+ df_all['sim_FCTR']+ df_all['sim_FGEV']

        clm_var = 'sim_Rnet'
        # -- Net Radiation:
        # -- Rnet = FSA-FIRA
        df_all [clm_var] = df_all ['sim_FSA']-df_all['sim_FIRA']

        clm_var = 'sim_NEE'
        # -- Net Ecosystem Exchange
        # -- NEE = GPP- (AR+HR)
        # -- It seems like the sign are opposite so we calculated negative NEE
        df_all [clm_var] = -(df_all ['sim_GPP']-(df_all['sim_AR']+df_all['sim_HR']))

        # convert NEE units from  umolm-2s-1 to gc/m2/s
        df_all ['NEE']= df_all ['NEE']*(12.01/1000000)
        df_all ['GPP']= df_all ['GPP']*(12.01/1000000)

        # -- convert gc/m2/s to gc/m2/day
        df_all ['NEE']= df_all['NEE']*60*60*24
        df_all ['sim_NEE']= df_all['sim_NEE']*60*60*24

        df_all ['GPP']= df_all['GPP']*60*60*24
        df_all ['sim_GPP']= df_all['sim_GPP']*60*60*24

        #-- extract year, month, day, hour information from time
        df_all['year'] = df_all['time'].dt.year
        df_all['month'] = df_all['time'].dt.month
        df_all['day'] = df_all['time'].dt.day
        df_all['hour'] = df_all['time'].dt.hour

        tzone_shift = ds_eval.TimeDiffUtcLt

        #convert to local time
        df_all['local_time']= df_all['time'] + datetime.timedelta(hours=int(tzone_shift))
        df_all['local_hour'] = df_all['local_time'].dt.hour


        df_all['site']=neon_site

        csv_out = join(csv_dir, "preprocessed_"+neon_site+"_"+year+".csv")
        df_all.to_csv(csv_out,index=False)
        
        end_site = time.time()
        print ("Extracting data for "+neon_site+" took : ", end_site-start_site, "s.")
        print ("Preprocessed file is saved as : ", csv_out)

    except Exception as e: 
        print (e)
        print ('THIS SITE FAILED:', neon_site)
        failed_sites.append(neon_site)
        pass

---------------------------
Extracting data for BART
All simulation files for all years: [ 1461 files]
Reading all simulation files took: 14.23722791671753 s.
All evaluation files for all years: [ 48 files]
Reading all observation files took: 2.4937686920166016 s.


100%|██████████| 10/10 [00:17<00:00,  1.73s/it]


Extracting data for BART took :  36.51193332672119 s.
Preprocessed file is saved as :  /glade/scratch/wwieder/preprocessed_neon_v2/preprocessed_BART_2021.csv
---------------------------
Extracting data for HARV
All simulation files for all years: [ 1461 files]
Reading all simulation files took: 12.821144580841064 s.
All evaluation files for all years: [ 48 files]
Reading all observation files took: 1.9814295768737793 s.


100%|██████████| 10/10 [00:17<00:00,  1.77s/it]


Extracting data for HARV took :  34.87820911407471 s.
Preprocessed file is saved as :  /glade/scratch/wwieder/preprocessed_neon_v2/preprocessed_HARV_2021.csv
---------------------------
Extracting data for BLAN
All simulation files for all years: [ 1461 files]
Reading all simulation files took: 12.958711862564087 s.
All evaluation files for all years: [ 48 files]
Reading all observation files took: 2.012259006500244 s.


...

Extracting data for BONA took :  34.22428560256958 s.
Preprocessed file is saved as :  /glade/scratch/wwieder/preprocessed_neon_v2/preprocessed_BONA_2021.csv
---------------------------
Extracting data for DEJU
All simulation files for all years: [ 1461 files]
Reading all simulation files took: 12.728009223937988 s.
All evaluation files for all years: [ 48 files]
Reading all observation files took: 2.14980411529541 s.


100%|██████████| 10/10 [00:25<00:00,  2.52s/it]


Extracting data for DEJU took :  42.90458583831787 s.
Preprocessed file is saved as :  /glade/scratch/wwieder/preprocessed_neon_v2/preprocessed_DEJU_2021.csv
---------------------------
Extracting data for HEAL
All simulation files for all years: [ 1461 files]
Reading all simulation files took: 13.702540159225464 s.
All evaluation files for all years: [ 48 files]
Reading all observation files took: 2.3866970539093018 s.


100%|██████████| 10/10 [00:18<00:00,  1.85s/it]


Extracting data for HEAL took :  37.48426580429077 s.
Preprocessed file is saved as :  /glade/scratch/wwieder/preprocessed_neon_v2/preprocessed_HEAL_2021.csv
---------------------------
Extracting data for PUUM
All simulation files for all years: [ 0 files]
no files to open
THIS SITE FAILED: PUUM


In [10]:
print ("Extracting data for ", len(failed_sites), "sites failed : ")
print (*failed_sites, sep=" \n")

Extracting data for  49 sites failed : 
BART 
HARV 
BLAN 
SCBI 
SERC 
DSNY 
JERC 
OSBS 
GUAN 
LAJA 
STEI 
TREE 
UNDE 
KONA 
KONZ 
UKFS 
GRSM 
MLBS 
ORNL 
DELA 
LENO 
TALL 
DCFS 
NOGP 
WOOD 
CPER 
RMNP 
STER 
CLBJ 
OAES 
YELL 
MOAB 
NIWO 
JORN 
SRER 
ONAQ 
ABBY 
WREF 
SJER 
SOAP 
TEAK 
TOOL 
BARR 
BONA 
DEJU 
HEAL 
PUUM 
NIWO 
PUUM
